In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

In [26]:
# Function to generate samples for loads and wind production
def generate_samples(num_samples, loads, sd, wind_limits, wind_shape):
    data = []
    sd = [mean * sd for mean in loads] # Standard deviation for loads

    for sample_num in range(1, num_samples + 1):
        for hour in range(24):  # Fixed to 24 hours per day
            
            # Generate loads using normal distribution
            load_sample = [
                round(np.clip(np.random.normal(mean, std), 0, mean * 2), 2) 
                for mean, std in zip(loads, sd)
            ]  # Clipping to avoid unrealistic negative values or extreme high values
            
            # Generate wind production using Weibull distribution (typically skewed as most wind clusters around lower speed)
            wind_sample = [
                round(min(np.random.weibull(wind_shape) * 0.42 * limit, limit), 2) 
                for limit in wind_limits
            ]
            
            # Combine into one row with [Sample Number, Hour, L1, L2, L3, W1, W2]
            row = [sample_num, hour] + load_sample + wind_sample
            data.append(row)
    
    return data

# Load L1-L3 and wind capacity W1-W2
loads = [150, 50, 150]  # Mean values for L1, L2, L3
wind_limits = [10, 30]  # Maximum capacity for W1 and W2

# Generate data for the samples
samples_data = generate_samples(num_samples=1000, loads=loads, sd=0.10, 
                                wind_limits=wind_limits, wind_shape=1.1)

# Create column names for the DataFrame
columns = ["Sample_Nr", "Hour", "L1", "L2", "L3", "W1", "W2"] # Sample_Nr ~ Day

# Create the DataFrame
samples_df = pd.DataFrame(samples_data, columns=columns)

# Save to CSV
samples_df.to_csv("sample_simon.csv", index=False)

# Display the updated DataFrame structure
samples_df.head(30)


,Sample_Nr,Hour,L1,L2,L3,W1,W2
0,1,0,163.19,48.11,145.84,2.56,0.38
1,1,1,149.53,48.60,147.01,1.44,17.03
2,1,2,136.01,54.32,156.26,1.48,7.88
3,1,3,167.61,47.98,155.69,4.43,7.48
4,1,4,141.72,46.06,141.56,2.42,0.67
5,1,5,153.41,52.80,153.48,2.16,3.92
6,1,6,153.64,53.51,127.04,7.69,13.75
7,1,7,156.31,48.35,144.81,3.30,13.31
8,1,8,165.03,52.16,160.57,7.14,2.50
9,1,9,157.12,46.15,150.99,5.97,3.64


In [27]:
# Load the data from the data folder
wind_forecast = samples_df[['Hour', 'W1', 'W2']].copy()
load = samples_df[['Hour', 'L1', 'L2','L3']].copy()
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv', delimiter=';')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv', delimiter=';')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv', delimiter=';')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv', delimiter=';')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv', delimiter=';')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv', delimiter=';')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv', delimiter=';')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv', delimiter=';')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv', delimiter=';')


In [28]:
# Define the input data class
class InputData:
    
    def __init__(
        self,
        wind_forecast: pd.DataFrame, 
        bus: pd.DataFrame,
        load: pd.DataFrame,
        max_prod: pd.DataFrame,
        min_prod: pd.DataFrame,
        min_down_time: pd.DataFrame,
        min_up_time: pd.DataFrame,
        prod_cost: pd.DataFrame,
        ramp_rate: pd.DataFrame,
        start_up_cost: pd.DataFrame,
        transmission_cap: pd.DataFrame
    ):
        self.time = range(len(wind_forecast))  #maybe define it with lenght of wind_production
        self.wind_forecast = wind_forecast
        self.bus = bus
        self.load = load
        self.max_prod = max_prod
        self.min_prod = min_prod
        self.min_down_time = min_down_time
        self.min_up_time = min_up_time
        self.prod_cost = prod_cost
        self.ramp_rate = ramp_rate
        self.start_up_cost = start_up_cost
        self.transmission_cap = transmission_cap
        self.M = 1000  # Penalty for having flexible demand
        
        


In [29]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [30]:
# Define the optimization model class

class EconomicDispatch():
        
        def __init__(self, input_data: InputData):
            self.data = input_data 
            self.variables = Expando()
            self.constraints = Expando() 
            self.results = Expando() 
            self._build_model() 
            
        def _build_variables(self):
            # one binary variable for the status of each generator
            self.variables.status = {
                (i, t): self.model.addVar(vtype=GRB.BINARY, 
                                            name='status_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # one variable for each generator for each time of the day
            self.variables.prod_gen = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.max_prod.iloc[i-1, 0], 
                                           name='generation_G{}_{}'.format(i, t)) 
                                           for i in range(1, len(self.data.max_prod)+1) 
                                           for t in self.data.time}
            
            # one variable for each wind generator for each time of the day
            self.variables.prod_wind = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.wind_forecast.iloc[t, i], 
                                            name='wind_generation_W{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.wind_forecast.iloc[0, :])) 
                                            for t in self.data.time}
            
            # one variable for each start-up cost for each generator
            self.variables.start_up_cost = {
                 (i, t): self.model.addVar(lb=0, 
                                            name='start_up_cost_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # add two slack variables to always make the model feasible, allowing the demand to be flexible
            self.variables.epsilon = {
                t: self.model.addVar(lb=0, name='epsilon_{}'.format(t)) 
                for t in self.data.time}
            self.variables.delta = {
                t: self.model.addVar(lb=0, name='delta_{}'.format(t))
                for t in self.data.time}
            
            
        def _build_constraints(self):
            # Minimum capacity of the generator
            self.constraints.min_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] >= self.data.min_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}
            # Maximum capacity of the generator
            self.constraints.max_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] <= self.data.max_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}

            # Power balance constraint
            self.constraints.power_balance = {
                t: self.model.addConstr(
                    gp.quicksum(self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1)) + 
                    gp.quicksum(self.variables.prod_wind[i, t] for i in range(1, len(self.data.wind_forecast.iloc[0, :]))) == 
                    gp.quicksum(self.data.load.iloc[t, i] for i in range(1, len(self.data.load.iloc[0, :]))) +
                    self.variables.epsilon[t] - self.variables.delta[t]
                ) for t in self.data.time}

            # Transmission capacity constraint


            #Start-up costs constraint
            self.constraints.start_up_cost = {
                (i, t): self.model.addConstr(
                    self.variables.start_up_cost[i, t] >= self.data.start_up_cost.iloc[i-1, 0] * (self.variables.status[i, t] - self.variables.status[i, t-1])
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.start_up_cost_0 = {
                i: self.model.addConstr(
                    self.variables.start_up_cost[i, 0] >= self.data.start_up_cost.iloc[i-1, 0] * self.variables.status[i, 0]
                ) for i in range(1, len(self.data.max_prod)+1)}
            
            # Ramping constraint
            # Arent we missing a constraint for hour 0 here? Or do we just assume we take over a running system?
            self.constraints.ramping_up = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] - self.variables.prod_gen[i, t-1] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.ramping_down = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t-1] - self.variables.prod_gen[i, t] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            
            # Minimum up time constraint
            self.constraints.min_up_time = {
                (i, t, to): self.model.addConstr(
                    -self.variables.status[i, t - 1] + self.variables.status[i, t] - self.variables.status[i, to] <= 0
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            # Minimum down time constraint
            self.constraints.min_down_time = {
                (i, t, to): self.model.addConstr(
                    self.variables.status[i, t - 1] - self.variables.status[i, t] + self.variables.status[i, to] <= 1
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            
            


            
            
            


        def _build_objective(self):
            # Objective function
            self.model.setObjective(
                gp.quicksum(self.data.prod_cost.iloc[i-1, 0]*self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                gp.quicksum(self.variables.start_up_cost[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                self.data.M * (gp.quicksum(self.variables.epsilon[t] for t in self.data.time) + gp.quicksum(self.variables.delta[t] for t in self.data.time))
            )

        def _build_model(self):
            self.model = gp.Model('EconomicDispatch')
            self._build_variables()
            self._build_constraints()
            self._build_objective()
            self.model.update()

        def optimize(self):
            self.model.optimize()
            self._extract_results()

        def _extract_results(self):
            self.results.production = pd.DataFrame({
                'time': [t for t in self.data.time],
                'status 1': [self.variables.status[1, t].x for t in self.data.time],
                'status 2': [self.variables.status[2, t].x for t in self.data.time],
                'status 3': [self.variables.status[3, t].x for t in self.data.time],
                'start_up_cost 1': [self.variables.start_up_cost[1, t].x for t in self.data.time],
                'start_up_cost 2': [self.variables.start_up_cost[2, t].x for t in self.data.time],
                'start_up_cost 3': [self.variables.start_up_cost[3, t].x for t in self.data.time],
                'generation 1': [self.variables.prod_gen[1, t].x for t in self.data.time],
                'generation 2': [self.variables.prod_gen[2, t].x for t in self.data.time],
                'generation 3': [self.variables.prod_gen[3, t].x for t in self.data.time],
                'wind generation 1': [self.variables.prod_wind[1, t].x for t in self.data.time],
                'wind generation 2': [self.variables.prod_wind[2, t].x for t in self.data.time],
                'load 1': [self.data.load.iloc[t, 1] for t in self.data.time],
                'load 2': [self.data.load.iloc[t, 2] for t in self.data.time],
                'load 3': [self.data.load.iloc[t, 3] for t in self.data.time],
                'epsilon': [self.variables.epsilon[t].x for t in self.data.time],
                'delta': [self.variables.delta[t].x for t in self.data.time]
            })
            
                
            
                 

In [31]:
# Run the model
input_data = InputData(wind_forecast, bus, load, max_prod, min_prod, min_down_time, min_up_time, prod_cost, ramp_rate, start_up_cost, transmission_cap)
model = EconomicDispatch(input_data)
model.optimize()
Results = model.results.production


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 743963 rows, 312000 columns and 1751904 nonzeros
Model fingerprint: 0xc041262a
Variable types: 240000 continuous, 72000 integer (72000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e-02, 2e+02]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 8.035272e+09
Presolve removed 312596 rows and 96413 columns (presolve time = 5s) ...
Presolve removed 312596 rows and 96413 columns
Presolve time: 6.05s
Presolved: 431367 rows, 215587 columns, 1198101 nonzeros
Variable types: 119588 continuous, 95999 integer (95999 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barr